In [1]:
!pip install requests pandas google-cloud-bigquery


EXTRACT DATA

In [23]:
import requests
import pandas as pd

def fetch_weather_data():

    weather_data = {
        'city': ['New York'],
        'date': ['2023-08-14'],
        'temperature': [25]
    }
    return pd.DataFrame.from_dict(weather_data)


def fetch_air_quality_data():

    air_quality_data = {
        'city': ['New York'],
        'date': ['2023-08-14'],
        'aqi': [50]
    }
    return pd.DataFrame.from_dict(air_quality_data)

weather_df = fetch_weather_data()
air_quality_df = fetch_air_quality_data()


TRANSFORM DATA

In [24]:
def transform_data(weather, air_quality):
    weather['date'] = pd.to_datetime(weather['date'])
    air_quality['date'] = pd.to_datetime(air_quality['date'])

 
    combined_df = pd.merge(weather, air_quality, on=['city', 'date'], how='inner')

    combined_df.ffill(inplace=True)
    
    return combined_df


transformed_df = transform_data(weather_df, air_quality_df)


Data Loading into BigQuery

In [25]:
from google.oauth2 import service_account
from google.cloud import bigquery

service_account_path = '/Users/fhariyaaseem/Downloads/web-data-418217-164309b13055.json'


credentials = service_account.Credentials.from_service_account_file(
    service_account_path
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

client = bigquery.Client(credentials=credentials)

In [31]:
# Correct format for project ID, dataset, and table names
project_id = 'web-data-418217'  # Your Google Cloud project ID
dataset_name = 'weather_data'  # Your dataset name
table_name = 'weather_air_quality'  # Your table name


full_dataset_id = f"{project_id}.{dataset_name}"


dataset_ref = client.dataset(dataset_name, project=project_id)


dataset = bigquery.Dataset(dataset_ref)


dataset.location = "US"


client.create_dataset(dataset, exists_ok=True)  # Make an API request.

# Define the schema for your table
schema = [
    bigquery.SchemaField("city", "STRING"),
    bigquery.SchemaField("date", "DATE"),
    bigquery.SchemaField("temperature", "FLOAT"),
    bigquery.SchemaField("aqi", "INTEGER"),
]


full_table_id = f"{project_id}.{dataset_name}.{table_name}"


table_ref = client.dataset(dataset_name).table(table_name)


table = bigquery.Table(table_ref, schema=schema)


client.create_table(table, exists_ok=True)  # Make an API request.


Table(TableReference(DatasetReference('web-data-418217', 'weather_data'), 'weather_air_quality'))

Load data into BigQuery

In [28]:
job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_TRUNCATE")
load_job = client.load_table_from_dataframe(transformed_df, table_ref, job_config=job_config)
load_job.result() 

print(f"Loaded {load_job.output_rows} rows into {full_table_id}.")


Loaded 1 rows into web-data-418217.weather_data.weather_air_quality.


In [16]:
!pip install db-dtypes

In [17]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/Users/fhariyaaseem/Downloads/web-data-418217-164309b13055.json'


In [19]:
from google.cloud import bigquery

client = bigquery.Client()


Average Temperature

In [43]:
from google.cloud import bigquery

client = bigquery.Client()

query = f"""
SELECT city, AVG(temperature) AS avg_temp
FROM `{full_table_id}`
GROUP BY city
"""
query_job = client.query(query) 

query_results_df = query_job.to_dataframe()

print(query_results_df)



       city  avg_temp
0  New York      25.0


Minimum and Maximum Temperature 

In [39]:
from google.cloud import bigquery

client = bigquery.Client()

query_yearly_temp_range = f"""
SELECT EXTRACT(YEAR FROM date) AS year, MAX(temperature) AS max_temp, MIN(temperature) AS min_temp
FROM `{full_table_id}`
GROUP BY year
ORDER BY year
"""
query_job_yearly_temp_range = client.query(query_yearly_temp_range)
df_yearly_temp_range = query_job_yearly_temp_range.to_dataframe()

print(df_yearly_temp_range)


   year  max_temp  min_temp
0  2023      25.0      25.0


Average AQI for Each Month Across All Cities

In [38]:
from google.cloud import bigquery

client = bigquery.Client()

query_monthly_avg_aqi = f"""
SELECT EXTRACT(YEAR FROM date) AS year, EXTRACT(MONTH FROM date) AS month, AVG(aqi) AS avg_aqi
FROM `{full_table_id}`
GROUP BY year, month
ORDER BY year, month
"""
query_job_monthly_avg_aqi = client.query(query_monthly_avg_aqi)
df_monthly_avg_aqi = query_job_monthly_avg_aqi.to_dataframe()

print(df_monthly_avg_aqi)


   year  month  avg_aqi
0  2023      8     50.0


Yearly Average Temperature Comparison for Nyc

In [37]:
from google.cloud import bigquery

client = bigquery.Client()

query_temp_trend_over_years = f"""
SELECT EXTRACT(YEAR FROM date) AS year, AVG(temperature) AS avg_temp
FROM `{full_table_id}`
GROUP BY year
ORDER BY year
"""
query_job_temp_trend_over_years = client.query(query_temp_trend_over_years)
df_temp_trend_over_years = query_job_temp_trend_over_years.to_dataframe()

print(df_temp_trend_over_years)

   year  avg_temp
0  2023      25.0
